In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.special import expit

In [2]:
data = pd.read_csv('test.csv') ## data import

In [3]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_raw = data[0:1000].T ## Transpose data
Y_raw = data_raw[0]
X_raw = data_raw[0:n]
X_raw = X_raw / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[0:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [4]:
def init_params(): ## set params in accordance with defined functions
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def RectLin(Z):
    return np.maximum(Z, 0)
    
def RectLinPrime(Z):
    ## derivative of the rectlin function, either 1 or 0 lol
    return Z > 0
    
def softmax(Z):
   # Ensure Z is a numpy array
    Z = np.array(Z)

    # Compute the softmax probabilities using expit
    A = expit(Z)
    return A
    
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def forward_prop(W1, b1, W2, b2, X):
    ## signal passthrough ~ lincomb from previous layers + activation function in order to create hidden layer
    Z1 = W1.dot(X) + b1
    A1 = RectLin(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    ## error rectification
    one_hot_Y = one_hot(Y)
    
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    
    dZ1 = W2.T.dot(dZ2) * RectLinPrime(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2


def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print("Accuracy: ", get_accuracy(predictions, Y))            
    return W1, b1, W2, b2
        

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 5000)

Iteration:  0
[7 2 7 ... 2 7 2] [0 0 0 ... 0 0 0]
Accuracy:  0.00037037037037037035
Iteration:  10
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.0015555555555555555
Iteration:  20
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.000925925925925926
Iteration:  30
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.000962962962962963
Iteration:  40
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.001037037037037037
Iteration:  50
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.001037037037037037
Iteration:  60
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.001037037037037037
Iteration:  70
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.001
Iteration:  80
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.001
Iteration:  90
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.000962962962962963
Iteration:  100
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.001074074074074074
Iteration:  110
[6 3 6 ... 7 6 2] [0 0 0 ... 0 0 0]
Accuracy:  0.001037037037037037
Iteration:  120
[6 3 6 ... 7 6

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)